In [8]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

import numpy as np

# Create Training Data

In [ ]:
xs = np.array([[ 0.0, 0.0, 0.0],
               [ 1.0, 1.0, 1.0],
               [-1.0,-1.0,-1.0],
               [ 1.0, 0.0, 0.0],
               [ 0.0, 1.0, 0.0],
               [ 0.0, 0.0, 1.0]])
ys = np.array([[1],
               [0],
               [0],
               [0],
               [0],
               [0]])

# Custom Layer 
computes whether input point is on the right side of the Plane

In [77]:
class InHull(tf.keras.layers.Layer):
    def __init__(self, units=1): # one output
        super().__init__()
        self.units = units
    
    def build(self, input_shape):
        w_init=tf.random_normal_initializer()
        nHplanes = input_shape[-1] + 1
        nDims = input_shape[-1]
        self.w = tf.Variable(initial_value = w_init(shape = (nHplanes, nDims),
                                                    dtype="float32"), 
                             trainable = True)
        print("Weights in layer: ", self.w)
        #print("length of layer: ", len(self.w)/2)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value = b_init(shape = (nHplanes, nDims),
                                                    dtype = "float32"), 
                             trainable = True)
        print("Bias in Layer: ", self.b)

    def call(self, inputs):
        # print("weights:", self.w)
        # print("biases:", self.b)
        # print("inputs:", inputs)
        # ntpoint = inputs
        # point = np.transpose(ntpoint)
        # print("point:", point)
        # dots = [K.sum(self.w[i] * (self.b[i] - point), axis = -1, keepdims = True) for i in range(4)]#range(len(self.w)/2)]
        # print("dots", dots)
        res = K.sum(self.w[0] * (self.w[1] - inputs), axis = -1, keepdims = True)
        print(res)
        return tf.Tensor(1.0)
        if res >= 0.0: res = 1
        else: res = 0
        return 1
        # return K.sum(self.w[0] * (self.w[1] - inputs), axis = -1, keepdims = True)
        # return K.batch_dot(inputs * self.w, axis =-1,keepdims = True) + self.b

# Custom Loss

In [34]:
def in_convex_hull(point, plane_point, plane_normal):
    if np.dot(plane, (point - a)) == 1:
        return true
    return false

def custom_loss(xs):
    def loss(y_true, y_pred):
        if y_pred >= 0:
            y_pred = 1.0
        else: y_pred = 0.0
        return K.square(y_true - y_pred)
    return loss

# Create Training Data

In [71]:
xs = np.array([[ 0.0, 0.0, 0.0],
               [ 1.0, 1.0, 1.0],
               [-1.0,-1.0,-1.0],
               [ 1.0, 0.0, 0.0],
               [ 0.0, 1.0, 0.0],
               [ 0.0, 0.0, 1.0]])
ys = np.array([[1],
               [0],
               [0],
               [0],
               [0],
               [0]])
# xs = np.array([[ 0.0, 0.0, 0.0]])
# ys = np.array([[1.0]])

# Compile Model and fit

In [78]:
model = keras.Sequential([InHull(1)])
model.compile(optimizer='Adam', loss='BinaryCrossentropy', metrics = ["accuracy"])
# model.compile(optimizer='Adam', loss=custom_loss(xs), metrics = ["accuracy"])


history = model.fit(xs, ys, epochs=10, batch_size=1, verbose = 0)
model.summary()

Weights in layer:  <tf.Variable 'in_hull_28/Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.03948059,  0.03049618,  0.02840161],
       [ 0.06689744, -0.0031126 ,  0.0107098 ],
       [-0.02699341, -0.00907688,  0.06186764],
       [-0.04331487, -0.03236471,  0.04431604]], dtype=float32)>
Bias in Layer:  <tf.Variable 'in_hull_28/Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>
Tensor("in_hull_28/Sum:0", shape=(1, 1), dtype=float32)
Tensor("sequential_28/in_hull_28/Sum:0", shape=(1, 1), dtype=float32)


TypeError: in user code:

    File "/home/thilo/miniconda3/envs/convexgating/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/thilo/miniconda3/envs/convexgating/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/thilo/miniconda3/envs/convexgating/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/thilo/miniconda3/envs/convexgating/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/thilo/miniconda3/envs/convexgating/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filefpg

In [76]:
xs_test = np.array([[ 0.0, 0.0, 0.0]])

model.predict(xs_test)

Tensor("in_hull_27/Sum:0", shape=(None, 1), dtype=float32)
Tensor("sequential_27/in_hull_27/Sum:0", shape=(None, 1), dtype=float32)
1/1 [==============================] - 0s 49ms/step


array([[0.00193928]], dtype=float32)

In [6]:
print("Weights: ", model.layers[0].get_weights()[0])
print("Biases: ", model.layers[0].get_weights()[1])

Weights:  [[ 0.04484339 -0.01542579  0.01255676]
 [ 0.02095187  0.01758021 -0.02743695]
 [-0.02538426  0.02119865  0.0322537 ]
 [-0.01514442  0.06497443 -0.01204634]]
Biases:  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [49]:
weight = tf.Variable([[-0.02217364,  0.02388768, -0.01642606],
       [ 0.0470141 , -0.1108605 ,  0.06211011],
       [ 0.00317579,  0.00729559,  0.13069701],
       [-0.0888112 ,  0.11896175, -0.07816519]])
bias = tf.Variable([[1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
input = tf.Variable([0.0,0.0,0.0])
point = input
print(weight, bias, point)
print("stop")
dots = tf.Variable([K.sum(weight[i] * (bias[i] - point), axis = -1, keepdims = True) for i in range(4)])#range(len(self.w)/2)]
# dots = dots/dots
print(dots[1] == 1)

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.02217364,  0.02388768, -0.01642606],
       [ 0.0470141 , -0.1108605 ,  0.06211011],
       [ 0.00317579,  0.00729559,  0.13069701],
       [-0.0888112 ,  0.11896175, -0.07816519]], dtype=float32)> <tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)> <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>
stop
tf.Tensor([False], shape=(1,), dtype=bool)


In [6]:
xs = np.array([np.random.rand(1000,3) for i in range(1)])[0]
print(xs)
xy = np.array( for x in xs)

[[0.88724768 0.7934657  0.6324761 ]
 [0.83078754 0.44004283 0.635758  ]
 [0.33647652 0.92324053 0.86827978]
 ...
 [0.18818799 0.73330666 0.39180892]
 [0.98370983 0.93480106 0.39973269]
 [0.12962338 0.76257981 0.94893442]]
